### Generate Liks

In [ ]:
import requests
from bs4 import BeautifulSoup as bs4

In [16]:
all_school_tests = "https://www.alloschool.com/course/"

In [17]:
pages = {"10": {"math": ["alriadhiat-aljtha-almshtrk-alom",
                         "alriadhiat-aljtha-almshtrk-tknoloji"],
                "physics-chemistry": ["alfiziaa-oalkimiaa-aljtha-almshtrk-alom",
                                      "alfiziaa-oalkimiaa-aljtha-almshtrk-tknoloji"],
                "biology": ["alom-alhiaa-oalardh-aljtha-almshtrk-alom"]},

         "11": {"math": ["alriadhiat-alaola-bak-alom-riadhia",
                         "alriadhiat-alaola-bak-alom-tjribia",
                         "alriadhiat-alaola-bak-alom-otknolojiat-khrbaiia",
                         "alriadhiat-alaola-bak-alom-otknolojiat-mikanikia"],
                "physics-chemistry": ["alfiziaa-oalkimiaa-alaola-bak-alom-riadhia",
                                      "alfiziaa-oalkimiaa-alaola-bak-alom-tjribia",
                                      "alfiziaa-oalkimiaa-alaola-bak-alom-otknolojiat-khrbaiia",
                                      "alfiziaa-oalkimiaa-alaola-bak-alom-otknolojiat-mikanikia"],
                "biology": ["alom-alhiaa-oalardh-alaola-bak-alom-riadhia",
                            "alom-alhiaa-oalardh-alaola-bak-alom-tjribia"]},

         "12": {"math": ["alriadhiat-althania-bak-alom-riadhia-awa",
                         "alriadhiat-althania-bak-alom-riadhia-b",
                         "alriadhiat-althania-bak-alom-fiziaiia",
                         "alriadhiat-althania-bak-alom-alhiaa-oalardh",
                         "alriadhiat-althania-bak-alom-zraaia",
                         "alriadhiat-althania-bak-alom-otknolojiat-khrbaiia",
                         "alriadhiat-althania-bak-alom-otknolojiat-mikanikia"],
                "physics-chemistry": ["alfiziaa-oalkimiaa-althania-bak-alom-riadhia-awa",
                                      "alfiziaa-oalkimiaa-althania-bak-alom-riadhia-b",
                                      "alfiziaa-oalkimiaa-althania-bak-alom-fiziaiia",
                                      "alfiziaa-oalkimiaa-althania-bak-alom-alhiaa-oalardh",
                                      "alfiziaa-oalkimiaa-althania-bak-alom-zraaia",
                                      "alfiziaa-oalkimiaa-althania-bak-alom-otknolojiat-khrbaiia-1",
                                      "alfiziaa-oalkimiaa-althania-bak-alom-otknolojiat-mikanikia"],
                "biology": ["alom-alhiaa-oalardh-althania-bak-alom-riadhia-awa",
                            "alom-alhiaa-oalardh-althania-bak-alom-fiziaiia",
                            "alom-alhiaa-oalardh-althania-bak-alom-alhiaa-oalardh",
                            "alom-alhiaa-oalardh-althania-bak-alom-zraaia"]}
        }

In [ ]:
from collections import defaultdict
from urllib.parse import urljoin
import json

links = defaultdict(list)
for grade in pages.keys():
  for subject in pages[grade].keys():
    for page in pages[grade][subject]:
      print(f"{grade}-{subject}-{page}")
      response = requests.get(urljoin(all_school_tests, str(page)))
      html = bs4(response.content)
      entries = html.find_all("li", {"class":"element"})
      question_entries = [ e for e in entries if e.find("span", {"class":"mdi"}) ]
      for question_entry in question_entries:
        link = question_entry.find("a")["href"]
        links[f"{grade}-{subject}"].append(link)

json.dump(links, open("all_school_link.json", "w", encoding="utf-8"))

### Generate QA Raw

In [19]:
import json
import requests
from bs4 import BeautifulSoup as bs4
from collections import defaultdict

def get_questions_payload(all_school_html_test_form):
  payload = {}
  payload["test_id"] = all_school_html_test_form.find("input", {"type":"hidden", "name":"test_id"})["value"]
  payload["time"] = all_school_html_test_form.find("input", {"type":"hidden", "name":"time"})["value"]
  payload["single_response"] = all_school_html_test_form.find("input", {"type":"hidden", "name":"single_response"})["value"]

  # Get all questions
  questions = all_school_html_test_form.find_all("div", {"class": "question-mcq"})

  # Collect ansersw name-value
  for question in questions:
    # Answers hidden variables
    question_answers = question.find_all("input", {"type":"hidden"})
    for question_answer in question_answers:
      payload[question_answer["name"]] = question_answer["value"]
  payload["submit"] = ""
  return payload

def process_question_answer(allschool_question_answer_html):
  question_answer = {}
  question_answer["question"] = allschool_question_answer_html.find("div", {"class": "question-question"}).decode_contents()
  question_answer["answer"]= {}
  options_div =  allschool_question_answer_html.find("div", {"class": "question-answers"})
  options =  options_div.find_all("div", recursive=False)
  correct_options = []
  for i,option in enumerate(options):
    question_answer["answer"][f"option_{i}"] = option.find("label").decode_contents()
    if option.find("span", {"class": "mdi-checkbox-marked-outline"}): correct_options.append(i)
  question_answer["answer"]["correct_options"] = correct_options
  return question_answer

In [ ]:
from collections import defaultdict
import json

total_question_answers= defaultdict(list)
links = json.load(open("all_school_link.json", "r"))
for grade_subject in links.keys():
  print(grade_subject)
  for link in links[grade_subject]:
    print(link)
    s = requests.Session()
    # Generate payload
    payload = {}
    response = s.get(link)
    if(response.status_code != 200):
      print("Failed!")
      continue
    html = bs4(response.content)
    form = html.find("form", {"class":"form-horizontal my-4"})
    payload = get_questions_payload(form)
    # Get Answers
    response = s.post("https://www.alloschool.com/test/submit", data=payload)
    html = bs4(response.content)
    answers = html.find_all("div", {"class": "question-mcq"})
    for answer_div in answers:
      answer = process_question_answer(answer_div)
      answer["link"]=link
      total_question_answers[grade_subject].append(answer)
    break
json.dump(total_question_answers, open("allschool_qa_raw.json", "w", encoding="utf-8"))

### Generate QA


In [21]:
from bs4 import BeautifulSoup
import re
from collections import defaultdict
import json

In [22]:
def filter_out(question_answer):
  filter_out_b = False
  mathjax = '<math'
  img = '<img'
  table = '<table'
  question = question_answer["question"]
  options = [v for k,v in question_answer["answer"].items() if "option" in k]
  for option in options:
    if mathjax in option or img in option or table in option:
      filter_out_b = True
      break
  if mathjax in question or img in question or table in question:
    filter_out_b= True
  return filter_out_b

In [ ]:
allschool_qa = json.load(open("allschool_qa_raw.json", encoding="utf-8"))

In [37]:
# propagate multi questions
fixed_qa= defaultdict(list)
for grade_subject in allschool_qa.keys():
  question_answers = allschool_qa[grade_subject]
  anchor_question = ""
  for question_answer in question_answers:
    title = BeautifulSoup(question_answer["question"]).find("u")
    order = re.search(r'(\d+)/(\d+)', title.text) if title else ""
    # if order:
    #   print(f"{title} - {order} - {order.group(1)}/{order.group(2)} - {question_answer['link']}")
    if order:
      if order.group(1)=='1':
        fixed_qa[grade_subject].append(question_answer)
        elements = BeautifulSoup(question_answer["question"])
        last_p_tag = elements.find_all('p')[-1].decompose()
        anchor_question = elements.decode_contents()
        # print(f"Anchor: {anchor_question}")
      else:
        question_answer["question"] = anchor_question + question_answer["question"]
        fixed_qa[grade_subject].append(question_answer)
        # print(f"Propagated: {question_answer['question']}")
    else:
      fixed_qa[grade_subject].append(question_answer)

In [39]:
# filter mathjax, img questions
filtered_qa= defaultdict(list)
for grade_subject in fixed_qa.keys():
  question_answers = fixed_qa[grade_subject]
  for question_answer in question_answers:
    if not filter_out(question_answer):
      filtered_qa[grade_subject].append(question_answer)

In [ ]:
# filter mcq with 2 or more options
filtered_qa_2= defaultdict(list)
for grade_subject in filtered_qa.keys():
  question_answers = filtered_qa[grade_subject]
  for question_answer in question_answers:
    if len(question_answer["answer"]["correct_options"])==1:
      filtered_qa_2[grade_subject].append(question_answer)

In [41]:
# textify content
textified_qa= defaultdict(list)
for grade_subject in filtered_qa_2.keys():
  question_answers = filtered_qa_2[grade_subject]
  for question_answer in question_answers:

    question = BeautifulSoup(question_answer["question"])
    _ = [u.decompose() for u in question.find_all('u')]
    _ = [s.decompose() for s in question.find_all('strong')]
    question_answer["question"] = question.text.strip()

    for k, v in  question_answer["answer"].items():
      if k == "correct_options":
        question_answer["answer"][k] = v[0] # set options to one value not array
        continue
      question_answer["answer"][k] = BeautifulSoup(v).text.strip()

    textified_qa[grade_subject].append(question_answer)

In [42]:
all_school_columnar=[]
for grade_subject in textified_qa.keys():
  grade, subject = grade_subject.replace("physics-chemistry", "physics_chemistry").split("-")
  for qa in textified_qa[grade_subject]:
    columnar = {}
    columnar["grade"] = grade
    columnar["subject"] = subject.replace("_", "-")
    columnar["question_text"] = qa["question"]
    columnar["option_0_text"] = qa["answer"]["option_0"] if "option_0" in qa["answer"] else ""
    columnar["option_1_text"] = qa["answer"]["option_1"] if "option_1" in qa["answer"] else ""
    columnar["option_2_text"] = qa["answer"]["option_2"] if "option_2" in qa["answer"] else ""
    columnar["option_3_text"] = qa["answer"]["option_3"] if "option_3" in qa["answer"] else ""
    columnar["correct_option"] = qa["answer"]["correct_options"]
    columnar["link"] = qa["link"]
    all_school_columnar.append(columnar)

In [44]:
import pandas as pd
all_school_df = pd.DataFrame(all_school_columnar)
# Remove Duplicates
all_school_df = all_school_df.drop_duplicates(subset=['question_text'])

In [45]:
questions = []
for i, row in all_school_df.iterrows():
  question = {}
  question["question"] = row["question_text"].strip()
  question["answer"] = {}
  for i in [0, 1, 2, 3]:
    question["answer"][f"option_{i}"] = row[f"option_{i}_text"]
  question["answer"][f"correct_option"] = row["correct_option"]
  question["grade"] = row["grade"]
  question["subject"] = row["subject"]
  question["resource"] = row["link"]
  questions.append(question)

In [46]:
questions[0]

{'question': 'القاسم المشترك الأكبر ل 1386 و 4620 هو',
 'answer': {'option_0': '770',
  'option_1': '462',
  'option_2': '154',
  'option_3': '110',
  'correct_option': 1},
 'grade': '10',
 'subject': 'math',
 'resource': 'https://www.alloschool.com/element/19994'}

In [ ]:
json.dump(questions, open("allschool_qa.json", "w", encoding="utf-8"), ensure_ascii=False)